Applying Optimization and Grid Search

In [1]:
#imports
import itertools
import pandas as pd

# Genetic Algorithm
from library.GA import genetic_algorithm, get_best_individual 

# Class GA Solution
from library.lineup import LUGASolution

# Mutation Functions
from library.mutation_funcs import block_rotation_mutation, two_phase_shuffle_mutation, semi_shuffle

# Crossover Functions
from library.xo_funcs import cyclic_crossover, custom_pmxo

# Selection Algorithm
from library.selection_algorithms import tournament_selection, ranking_selection
from library.selection_algorithms import fitness_proportionate_selection #NÃO PODEMOS USAR ERA SÓ PARA EXEMPLO

# Line-up Problem: Datasets
from library.lineup import artists_df, conflicts_df, HERE

In [ ]:
POP_SIZE = 10

initial_population = [LUGASolution(mutation_function = block_rotation_mutation, crossover_function = cyclic_crossover
                                   )
                      for i in range(POP_SIZE)]

genetic_algorithm(initial_population=initial_population,
                  max_gen=10,
                  selection_algorithm = tournament_selection,
                  maximization=True,
                  xo_prob=0.8,
                  mut_prob=1,
                  elitism=True,
                  verbose=False)[0]

Generations:   0%|          | 0/10 [00:00<?, ?gen/s]

Gen  1:   0%|          | 0/10 [00:00<?, ?ind/s]

<LUSolution (5 Stages, 7 Slots)>
╒═════════╤══════════════════════════╤══════════════════╤════════════════════╤═══════════════════════╤═════════════════════════╤════════════════╤════════════════════╕
│         │          Slot 1          │      Slot 2      │       Slot 3       │        Slot 4         │         Slot 5          │     Slot 6     │       Slot 7       │
╞═════════╪══════════════════════════╪══════════════════╪════════════════════╪═══════════════════════╪═════════════════════════╪════════════════╪════════════════════╡
│ Stage 1 │      Shadow Cadence      │ The Jazz Nomads  │    Astral Tide     │    Crimson Harmony    │ The Bassline Architects │ Static Mirage  │ Electric Serpents  │
├─────────┼──────────────────────────┼──────────────────┼────────────────────┼───────────────────────┼─────────────────────────┼────────────────┼────────────────────┤
│ Stage 2 │       Golden Ember       │  Rhythm Alchemy  │ The Sonic Drifters │ Cloud Nine Collective │      Midnight Echo      │  Tu

In [41]:
def comb_tuning(mut_func: dict, xo_func: dict, selection_algo: list,
                n_runs: int = 30, max_gen:int = 100, pop_size: int = 50):
    
    """
    Returns a DataFrame with the following columns:
    - 'Combination' - Combination tuple of a mutation, a crossover and a selection algorithm
    - 'Run' - Each Combination is runned a specified amount of times, this column specifies the run 
    number in which the results were obtained.
    - 'Generation' - Each Run is composed of a specified amount of generations, this column specifies the 
    generation number in which the results were obtained.
    - 'Best Fitness' - This is the best fitness obtained at each generation.

    Purpose:
    With this df we can play around inter-run and intra-run statistics, since it contains 
    all the results from the various combinations
    """

    combinations = list(itertools.product(list(mut_func.keys()), list(xo_func.keys()), selection_algo))

    combs_results = []
    for comb in combinations:
        # Store the best results of each generation for each run.
        runs_results = []
        for run in range(1, n_runs+1):
            initial_population = [LUGASolution(mutation_function = comb[0], crossover_function = comb[1])
                                               for i in range(pop_size)]
            
            _, generation_df = genetic_algorithm(initial_population=initial_population,
                    max_gen=max_gen,
                    selection_algorithm = comb[2],
                    maximization=True,
                    xo_prob=xo_func[comb[1]],
                    mut_prob= mut_func[comb[0]],
                    elitism=True,
                    verbose=False)
            
            current_run = pd.DataFrame()
            current_run['Run'] = [run for _ in range(1, max_gen+1)] #fill column 'Run' with the #run it is in
            
            #with run=#run, merge the df that contains the columns with #generation and its best fitness
            one_run = pd.concat([current_run, generation_df], axis=1) 
            runs_results.append(one_run)
        
        current_comb_all_runs = pd.concat(runs_results) #concat all the different runs along the index axis
        
        current_comb = pd.DataFrame()
        current_comb['Combination'] = [(comb[0].__name__, comb[1].__name__, comb[2].__name__)
                                       for _ in range(current_comb_all_runs.shape[0])]
        one_comb = pd.concat([current_comb.reset_index(drop=True), current_comb_all_runs.reset_index(drop=True)], axis=1)
        combs_results.append(one_comb)
    
    global final_results
    final_results = pd.concat(combs_results)
    final_filepath = HERE.parent / "combination_search"/ "final_results.csv"
    final_results.to_csv(final_filepath, index=False)
    
    return final_results

In [ ]:
#Example
mut_func = {block_rotation_mutation:0.07, two_phase_shuffle_mutation: 0.1, semi_shuffle: 0.2}
xo_func = {cyclic_crossover:0.7, custom_pmxo: 0.8}
selection_algo = [tournament_selection]#, fitness_proportionate_selection]

In [52]:
#experimentar 1ª com poucas runs e poucas gen e pop baixa para ver se corre tudo bem
comb_tuning(mut_func = mut_func, xo_func = xo_func, selection_algo=selection_algo,
            n_runs=2, max_gen=2, pop_size=5)

Generations:   0%|          | 0/2 [00:00<?, ?gen/s]

Gen  1:   0%|          | 0/5 [00:00<?, ?ind/s]

Generations:   0%|          | 0/2 [00:00<?, ?gen/s]

Gen  1:   0%|          | 0/5 [00:00<?, ?ind/s]

Generations:   0%|          | 0/2 [00:00<?, ?gen/s]

...

Generations:   0%|          | 0/2 [00:00<?, ?gen/s]

Gen  1:   0%|          | 0/5 [00:00<?, ?ind/s]

Generations:   0%|          | 0/2 [00:00<?, ?gen/s]

Gen  1:   0%|          | 0/5 [00:00<?, ?ind/s]

,Combination,Run,Generation,Fitness
0,"(block_rotation_mutation, cyclic_crossover, to...",1,1,1.16667
1,"(block_rotation_mutation, cyclic_crossover, to...",1,2,1.16667
2,"(block_rotation_mutation, cyclic_crossover, to...",2,1,1.01143
3,"(block_rotation_mutation, cyclic_crossover, to...",2,2,1.07796
0,"(block_rotation_mutation, cyclic_crossover, fi...",1,1,1.10181
1,"(block_rotation_mutation, cyclic_crossover, fi...",1,2,1.10181
2,"(block_rotation_mutation, cyclic_crossover, fi...",2,1,1.04867
3,"(block_rotation_mutation, cyclic_crossover, fi...",2,2,1.04867
0,"(block_rotation_mutation, custom_pmxo, tournam...",1,1,1.16733
1,"(block_rotation_mutation, custom_pmxo, tournam...",1,2,1.16733


In [60]:
results = pd.read_csv(HERE.parent /'combination_search' /'final_results.csv')
results

,Combination,Run,Generation,Fitness
0,"('block_rotation_mutation', 'cyclic_crossover'...",1,1,1.16667
1,"('block_rotation_mutation', 'cyclic_crossover'...",1,2,1.16667
2,"('block_rotation_mutation', 'cyclic_crossover'...",2,1,1.01143
3,"('block_rotation_mutation', 'cyclic_crossover'...",2,2,1.07796
4,"('block_rotation_mutation', 'cyclic_crossover'...",1,1,1.10181
5,"('block_rotation_mutation', 'cyclic_crossover'...",1,2,1.10181
6,"('block_rotation_mutation', 'cyclic_crossover'...",2,1,1.04867
7,"('block_rotation_mutation', 'cyclic_crossover'...",2,2,1.04867
8,"('block_rotation_mutation', 'custom_pmxo', 'to...",1,1,1.16733
9,"('block_rotation_mutation', 'custom_pmxo', 'to...",1,2,1.16733


In [ ]:
# It is also stored in a global variable, not only in csv

#unique = final_results['Combination'].unique()
#for u in unique:
#    print(final_results[final_results['Combination']==u])